# <div align="center" style="color: #ff5733;">RBP Pilot Raw Data</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
sq = """WITH 
W_VAS_FEES
AS
(
  select loanId, sum(scheduleChargeAmount) as VasFees  from prj-prod-dataplatform.dl_loans_db_raw.tdbk_loan_repayment_schedule  f
  group by 1
),
W_PROMOTERS
AS
(
  SELECT DISTINCT a.digitalLoanAccountId, a.promoterId, 
  #a.promoterName,
  b.firstName || Case when b.middleName is not null then ' ' || b.middleName End  || Case when b.lastName is not null then ' ' || b.lastName End as promoterName,
  CASE WHEN C.Date_of_leaving = DATE '3000-01-01' THEN NULL ELSE C.Date_of_leaving END AS Promoter_LWD, 
  row_number() over(partition by a.digitalLoanAccountId order by a.id desc) as rnk  
  FROM
  prj-prod-dataplatform.dl_loans_db_raw.tdbk_qr_promoter_details a
  left join prj-prod-dataplatform.dl_dynamo_db_raw.tdbk_promotors_user_dtl  b
  on a.promoterId = b.id
  LEFT JOIN prj-prod-dataplatform.dap_ds_poweruser_playground.SIL_Promoters_DOJ C
  ON TRIM(C.Promoter_ID) = TRIM(b.id)
  WHERE
  a.promoterId IS NOT NULL
)
,
W_LOAN_MASTER_a as
(
    SELECT a.digitalLoanAccountId, a.startApplyDateTime, a.purposeDescription,
    CASE 
    WHEN a.cddInitiatedDateAndTime IS NOT NULL AND a.cddVerificationDateAndTime IS NOT NULL THEN a.cddVerificationDateAndTime
    WHEN a.cddInitiatedDateAndTime IS NULL AND COALESCE(a.rejectedDateTime,a.approvedDateTime,a.Stage1_DateTime) IS NOT NULL THEN COALESCE(a.rejectedDateTime,a.approvedDateTime,a.Stage1_DateTime)
    ELSE NULL END AS decision_date, 
    a.disbursementDateTime, a.purpleKey, a.merchantName,
    d.BrandStores,
    a.stage1status, a.approvedLoanAmount, a.downPaymentAmount,
    (coalesce(cast(c.price as float64),0)) as Items_Sold_Price,
    a.isTermsAndConditionsAccepted, 
    CASE 
    WHEN a.cddInitiatedDateAndTime IS NOT NULL AND a.cddVerificationDateAndTime IS NOT NULL THEN a.flagApproval
    WHEN a.cddInitiatedDateAndTime IS NULL AND COALESCE(a.rejectedDateTime,a.approvedDateTime,a.Stage1_DateTime) IS NOT NULL AND a.stage1status in ('EXPIRED','APPROVED') THEN 1 
    ELSE 0 END AS flagApproval, 
    CASE 
    WHEN a.cddInitiatedDateAndTime IS NOT NULL AND a.cddVerificationDateAndTime IS NOT NULL THEN flagRejection
    WHEN a.cddInitiatedDateAndTime IS NULL AND COALESCE(a.rejectedDateTime,a.approvedDateTime,a.Stage1_DateTime) IS NOT NULL AND COALESCE(a.stage1status,'XNA') NOT in ('EXPIRED','APPROVED') THEN 1 
    ELSE 0 END AS flagRejection, 
    a.flagDisbursement, a.disbursedLoanTenur, a.disbursedLoanAmount, a.BNPL_PRODUCT_CATEGORY_CNT, a.employmentstatus, a.new_loan_type,
    B.promoterId, 
    coalesce(initcap(b.promoterName) ,'XNA') as Promoter,
    b.Promoter_LWD as Promoter_LWD,
    d.TSM, d.DateOpened,
    case when e.vas_flag = 'true' then 1 else 0 end as VasFlag,
    #f.scheduleChargeAmount as VasFees
    f.VasFees,
    FROM 
    prj-prod-dataplatform.risk_credit_mis.loan_master_table a
    left join W_PROMOTERS b
    on a.digitalLoanAccountId = b.digitalLoanAccountId and b.rnk = 1
    left join prj-prod-dataplatform.dl_loans_db_raw.tdbk_bnpl_trans_category c 
    on a.digitalLoanAccountId = c.digitalLoanAccountId
    left join prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING d
    on d.purpleKey = a.purpleKey
    left join prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application e
    on e.loanAccountNumber = a.loanAccountNumber
    left join W_VAS_FEES  f
    on a.loanAccountNumber = f.loanId# and f.repaymentNumber = 1
    WHERE
    a.new_loan_type = 'SIL-Instore'
    AND
    DATE_TRUNC(a.startApplyDateTime,DAY) BETWEEN DATE '2022-03-26' AND (CURRENT_DATE - 1)
  )
, W_LOAN_MASTER
AS
(
  SELECT digitalLoanAccountId, startApplyDateTime, DECISION_DATE, disbursementDateTime, purpleKey, BrandStores, merchantName, purposeDescription as Commodity, 
  case when stage1status in ('EXPIRED','APPROVED') then approvedLoanAmount end as approvedLoanAmount,
  case when stage1status in ('EXPIRED','APPROVED') then CAST(downPaymentAmount AS FLOAT64) end as downPaymentAmount,
  #approvedLoanAmount, 
  COALESCE(isTermsAndConditionsAccepted,0) AS APP_COMPLETED, 
  flagApproval,
  CASE WHEN DECISION_DATE IS NOT NULL AND flagApproval =1 AND flagDisbursement = 0 AND stage1status in ('APPROVED') THEN 1 ELSE 0 END AS APPLICATION_APPROVED_NOT_BOOKED_CNT,
  CASE WHEN DECISION_DATE IS NOT NULL AND flagApproval =1 AND flagDisbursement = 0 AND stage1status in ('EXPIRED') THEN 1 ELSE 0 END AS APPLICATION_EXPIRED,
  flagRejection,
  flagDisbursement, disbursedLoanTenur, disbursedLoanAmount, 
  CASE WHEN COALESCE(BNPL_PRODUCT_CATEGORY_CNT,0) > 1 THEN 'Y' Else 'N' END AS BUNDLED,
  Initcap(employmentstatus) AS employmentstatus, promoterId, Promoter, Promoter_LWD, 
  TSM, DateOpened,
  sum(case when stage1status in ('EXPIRED','APPROVED') then Items_Sold_Price end) as Items_Sold_Price,
  max(VasFlag) as VasFlag ,
  max(VasFees) as VasFees  from W_LOAN_MASTER_a
    WHERE
  new_loan_type = 'SIL-Instore'
  #loanType = 'BNPL' AND purpleKey is not null and (isUserAtStore = 1 or isUserAtStore is null)
  AND
  DATE_TRUNC(startApplyDateTime,DAY) BETWEEN DATE '2022-03-26' AND (CURRENT_DATE - 1)
  Group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24, 25
)
,
W_DTS
AS
(
    SELECT DISTINCT DATE_TRUNC(startApplyDateTime,MONTH) AS MONTH, DATE_TRUNC(startApplyDateTime,DAY) dt,
    case when DATE_TRUNC(startApplyDateTime,DAY) between '2024-02-09' and '2024-03-09' then 'B'
         when DATE_TRUNC(startApplyDateTime,DAY) > '2024-03-09' then 'A'
         when DATE_TRUNC(startApplyDateTime,DAY) < '2024-02-09' then 'C' end date_flag
    FROM prj-prod-dataplatform.risk_credit_mis.loan_master_table 
    WHERE
    DATE_TRUNC(startApplyDateTime,DAY) BETWEEN DATE '2022-03-26' AND (CURRENT_DATE - 1)
)
,
delq_base as
(select lmt.loanAccountNumber, lmt.digitalLoanAccountId,
sum(case when ldd.min_inst_def30 = 1 then 1 else 0 end) fpd30_cnt,
sum(case when ldd.obs_min_inst_def30 >= 1 then 1 else 0 end) obs_fpd30_cnt,
sum(case when ldd.min_inst_def30 = 1 then lmt.disbursedloanamount else 0 end) fpd30_v,
sum(case when ldd.obs_min_inst_def30 >= 1 then lmt.disbursedloanamount else 0 end) obs_FPD30_v,
sum(case when ldd.min_inst_def30 = 2 then 1 else 0 end) spd30_cnt,
sum(case when ldd.obs_min_inst_def30 >= 2 then 1 else 0 end) obs_spd30_cnt,
sum(case when ldd.min_inst_def30 = 2 then lmt.Total_Outstanding_Principal else 0 end) spd30_v,
sum(case when ldd.obs_min_inst_def30 >= 2 then lmt.disbursedloanamount else 0 end) obs_spd30_v,
sum(case when ldd.min_inst_def30 = 3 then 1 else 0 end) tpd30_cnt,
sum(case when ldd.obs_min_inst_def30 >= 3 then 1 else 0 end) obs_tpd30_cnt,
sum(case when ldd.min_inst_def30 = 3 then lmt.Total_Outstanding_Principal else 0 end) tpd30_v,
sum(case when ldd.obs_min_inst_def30 >=3 then lmt.disbursedloanamount else 0 end) obs_tpd30_v,
sum(case when ldd.obs_min_inst_def30 >= 3 and ldd.min_inst_def30 in (1,2,3) then 1 else 0 end) fstpd30_cnt,
sum(Case when ldd.obs_min_inst_def30 >= 3 then 1 else 0 end) obs_fstpd30_cnt
from
`risk_credit_mis.loan_master_table` lmt
left join `prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data` ldd on ldd.loanAccountNumber = lmt.loanAccountNumber
where lmt.flagDisbursement = 1
group by 1,2
),
loanbase as 
(select a.digitalLoanAccountId, a.startApplyDateTime, 
a.DECISION_DATE,
a.disbursementDateTime,
a.purpleKey,
case when a.purpleKey in ('MV5BS04', 'MD1R4BJ', 'M6EKLNI', 'MYXLHQ4', 'MHZDIVT', 'M2JQP9L', 'MYRK3EJ', 'M7SE4LZ', 'MDGYHQJ', 'MQTA03J', 'M7261RR', 'MR0G4XP', 'M8A59CP', 'M7ZLTH1', 'MWG98DQ', 'MA296F1') then 'RBP Pilot' else 'Other' end RBP_Flag,
a.merchantName,
a.BrandStores,
a.approvedLoanAmount,
a.downPaymentAmount,
a.APP_COMPLETED,
a.flagApproval,
a.APPLICATION_APPROVED_NOT_BOOKED_CNT,
a.APPLICATION_EXPIRED,
a.flagRejection,
a.flagDisbursement,
a.disbursedLoanTenur,
a.disbursedLoanAmount,
a.BUNDLED,
a.employmentstatus,
a.promoterId,
a.TSM,
a.DateOpened,
a.Items_Sold_Price,
a.VasFlag,
a.VasFees,
db.loanAccountNumber,
db.fpd30_cnt,
db.obs_fpd30_cnt,
db.fpd30_v,
db.obs_FPD30_v,
db.spd30_cnt,
db.obs_spd30_cnt,
db.spd30_v,
db.obs_spd30_v,
db.tpd30_cnt,
db.obs_tpd30_cnt,
db.tpd30_v,
db.obs_tpd30_v,
db.fstpd30_cnt,
db.obs_fstpd30_cnt
from W_LOAN_MASTER a
left join delq_base db on db.digitalLoanAccountId = a.digitalLoanAccountId
)
,
W_MERCHANTS
AS
(
    SELECT DISTINCT purpleKey AS MERCHANT_CODE, merchantName as Store, 
    BrandStores Merchant,
    CASE WHEN purplekey IN ('M9ZK6IU','MV4UO0X','M2G78K4','MEKA38B','MOE3JQH') THEN 'N' ELSE 'Y' END AS ACTIVE,
    COALESCE(TSM,'XNA') AS TSM, DateOpened, 
    BUNDLED, employmentstatus, promoterId
    FROM loanbase
),
W_STARTED
AS
(
    SELECT date(format_date('%Y-%m-%d',DATE_TRUNC(startApplyDateTime,MONTH))) AS APP_START_DATE,
    DATE_TRUNC(startApplyDateTime,DAY) dt,
    case when DATE_TRUNC(startApplyDateTime,DAY) between '2024-02-09' and '2024-03-09' then 'B'
         when DATE_TRUNC(startApplyDateTime,DAY) > '2024-03-09' then 'A'
         when DATE_TRUNC(startApplyDateTime,DAY) < '2024-02-09' then 'C' end date_flag,
    purpleKey, employmentstatus
    , max(merchantName) merchantName
    , max(RBP_Flag) RBP_Flag 
    ,COUNT(DISTINCT BrandStores) Merchant,
    min(DateOpened) pos_startofoperation,
    count(distinct  TSM) TSM,
    count(distinct promoterId) Promoters,
    COUNT(1) AS APP_STARTED, SUM(APP_COMPLETED) AS APP_COMPLETED, 
    #SUM(CASE WHEN DECISION_DATE IS NOT NULL AND flagApproval =1 AND flagDisbursement = 0 THEN 1 ELSE 0 END) AS APPLICATION_APPROVED_NOT_BOOKED_CNT,
    SUM(APPLICATION_APPROVED_NOT_BOOKED_CNT) AS APPLICATION_APPROVED_NOT_BOOKED_CNT,
    SUM(APPLICATION_EXPIRED) AS APPLICATION_EXPIRED,
    SUM(CASE WHEN DECISION_DATE IS NOT NULL AND flagApproval =1 AND flagDisbursement = 0 THEN approvedLoanAmount ELSE 0 END) AS APPROVED_NOT_BOOKED_VOLUME
    FROM loanbase
    WHERE startApplyDateTime IS NOT NULL
    GROUP BY 1,2,3,4,5
)
,
W_DECIDED
AS
(
    SELECT 
    date(format_date('%Y-%m-%d',DATE_TRUNC(DECISION_DATE,MONTH))) AS APP_DECISION_DATE,
    DATE_TRUNC(DECISION_DATE,DAY) dt,
    case when DATE_TRUNC(DECISION_DATE,DAY) between '2024-02-09' and '2024-03-09' then 'B'
         when DATE_TRUNC(DECISION_DATE,DAY) > '2024-03-09' then 'A'
         when DATE_TRUNC(DECISION_DATE,DAY) < '2024-02-09' then 'C' end date_flag
    , purpleKey, employmentstatus
    , max(merchantName) merchantName
    ,max(RBP_Flag) RBP_Flag,
    COUNT(DISTINCT BrandStores) Merchant,
    min(DateOpened) pos_startofoperation,
    count(distinct  TSM) TSM,
    count(distinct promoterId) Promoters,
    COUNT(1) AS APP_DECIDED, SUM(flagApproval) AS APP_APPROVED, SUM(flagRejection) AS APP_REJECTED, 
    SUM(approvedLoanAmount) AS approvedLoanAmount, SUM(downPaymentAmount) AS downPaymentAmount, SUM(Items_Sold_Price) AS Items_Sold_Price
    FROM loanbase
    WHERE DECISION_DATE IS NOT NULL
    GROUP BY 1,2,3,4,5
)
,
W_BOOKED
AS
(
    SELECT 
    date(format_date('%Y-%m-%d',DATE_TRUNC(disbursementDateTime,MONTH))) AS APP_BOOKED_DATE,
    DATE_TRUNC(disbursementDateTime,DAY) dt,
    case when DATE_TRUNC(disbursementDateTime,DAY) between '2024-02-09' and '2024-03-09' then 'B'
         when DATE_TRUNC(disbursementDateTime,DAY) > '2024-03-09' then 'A'
         when DATE_TRUNC(disbursementDateTime,DAY) < '2024-02-09' then 'C' end date_flag
    , purpleKey
    , employmentstatus
    , max(merchantName) merchantName
    , max(RBP_Flag) RBP_Flag
    ,COUNT(DISTINCT BrandStores) Merchant,
    count(distinct  TSM) TSM,
    count(distinct promoterId) Promoters,
    SUM(flagDisbursement) AS APP_BOOKED, SUM(disbursedLoanAmount) AS APP_BOOKED_VOLUME, 
    SUM(disbursedLoanTenur) AS APP_BOOKED_TENURE, 
    SUM(CASE WHEN disbursedLoanTenur > 3 THEN 1 ELSE 0 END) AS VAS_ELIGIBLE_BASE,
    SUM(VasFlag) AS VAS_SOLD, SUM(VasFees) AS VAS_SOLD_VOLUME,
    SUM(fpd30_cnt) fpd30_cnt,
    SUM(obs_fpd30_cnt) obs_fpd30_cnt,
    SUM(fpd30_v) fpd30_v,
    SUM(obs_FPD30_v) obs_FPD30_v,
    SUM(spd30_cnt) spd30_cnt,
    sum(obs_spd30_cnt) obs_spd30_cnt,
    SUM(spd30_v) spd30_v,
    SUM(obs_spd30_v) obs_spd30_v,
    SUM(tpd30_cnt) tpd30_cnt,
    SUM(obs_tpd30_cnt)obs_tpd30_cnt,
    SUM(tpd30_v) tpd30_v,
    SUM(obs_tpd30_v) obs_tpd30_v,
    SUM(fstpd30_cnt) fstpd30_cnt,
    SUM(obs_fstpd30_cnt) obs_fstpd30_cnt
    FROM loanbase
    WHERE disbursementDateTime IS NOT NULL
    GROUP BY 1,2,3,4,5
)
,
b as 
(select format_date('%Y-%m',W_DTS.MONTH)MONTH,
format_date('%Y-%m-%d', W_DTS.dt) dt, W_DTS.date_flag
, W_STARTED.purpleKey
, W_STARTED.employmentstatus
, max(W_STARTED.merchantName)merchantName
, max(W_STARTED.RBP_Flag) RBP_Flag
, SUM(W_STARTED.Merchant) Merchant
, min(W_STARTED.pos_startofoperation) pos_startofoperations
, sum(W_STARTED.TSM) TSM
, sum(W_STARTED.Promoters) Promoters
, sum(W_STARTED.APP_STARTED) APP_STARTED
, SUM(W_STARTED.APP_COMPLETED) AS APP_COMPLETED
, SUM(W_STARTED.APPLICATION_APPROVED_NOT_BOOKED_CNT) AS APPLICATION_APPROVED_NOT_BOOKED_CNT
, SUM(W_STARTED.APPLICATION_EXPIRED) AS APPLICATION_EXPIRED
, SUM(W_STARTED.APPROVED_NOT_BOOKED_VOLUME) APPROVED_NOT_BOOKED_VOLUME
, COALESCE(SUM(W_DECIDED.APP_DECIDED),0) AS APPLICATION_DECISION_CNT
, COALESCE(SUM(W_DECIDED.APP_APPROVED),0) AS APPLICATION_APPROVED_CNT
, COALESCE(SUM(W_DECIDED.APP_REJECTED),0) AS APPLICATION_REJECTED_CNT
, COALESCE(SUM(W_DECIDED.approvedLoanAmount),0) AS approvedLoanAmount 
, COALESCE(SUM(W_DECIDED.downPaymentAmount),0) AS downPaymentAmount
, COALESCE(SUM(W_DECIDED.Items_Sold_Price),0) AS Items_Sold_Price
, COALESCE(SUM(W_BOOKED.APP_BOOKED),0) AS APPLICATION_BOOKING_CNT
, COALESCE(SUM(W_BOOKED.APP_BOOKED_VOLUME),0) AS BOOKED_VOLUME
, COALESCE(SUM(W_BOOKED.APP_BOOKED_TENURE),0) AS APPLICATION_BOOKED_TENURE
, COALESCE(SUM(W_BOOKED.VAS_ELIGIBLE_BASE),0) AS VAS_ELIGIBLE_BASE
, COALESCE(SUM(W_BOOKED.VAS_SOLD),0) AS VAS_SOLD
, COALESCE(SUM(W_BOOKED.VAS_SOLD_VOLUME),0) AS VAS_SOLD_VOLUME
, COALESCE(SUM(W_BOOKED.fstpd30_cnt), 0) AS FSTPD30_CNT
, COALESCE(SUM(W_BOOKED.obs_fstpd30_cnt), 0) AS OBS_FSTPD30_CNT
, COALESCE(SUM(W_BOOKED.fpd30_cnt), 0) AS FPD30_CNT
, COALESCE(SUM(W_BOOKED.obs_fpd30_cnt), 0) AS OBS_FPD30_CNT
, COALESCE(SUM(W_BOOKED.fpd30_v), 0) AS FPD30_V
, COALESCE(SUM(W_BOOKED.obs_FPD30_v), 0) AS OBS_FPD30_V
, COALESCE(SUM(W_BOOKED.spd30_cnt), 0) AS SPD30_CNT
, COALESCE(SUM(W_BOOKED.obs_spd30_cnt), 0) AS OBS_SPD30_CNT
, COALESCE(SUM(W_BOOKED.tpd30_cnt), 0) as TPD30_CNT
, COALESCE(SUM(W_BOOKED.obs_tpd30_cnt), 0) AS OBS_TPD30_CNT
from W_DTS
LEFT JOIN W_STARTED ON W_STARTED.APP_START_DATE = W_DTS.MONTH AND W_STARTED.dt = W_DTS.dt and W_STARTED.date_flag = W_DTS.date_flag
LEFT JOIN W_DECIDED ON W_DECIDED.APP_DECISION_DATE = W_DTS.MONTH 
    AND W_DECIDED.dt = W_DTS.dt
    AND W_DECIDED.date_flag = W_DTS.date_flag
    AND COALESCE(W_DECIDED.purpleKey, 'XNA') =  COALESCE(W_STARTED.purpleKey, 'XNA') 
    AND COALESCE(W_DECIDED.employmentstatus, 'XNA') =  COALESCE(W_STARTED.employmentstatus, 'XNA') 
LEFT JOIN W_BOOKED ON W_BOOKED.APP_BOOKED_DATE = W_DTS.MONTH 
    AND W_BOOKED.dt = W_DTS.dt
    AND W_BOOKED.date_flag = W_DTS.date_flag
    AND COALESCE(W_DECIDED.purpleKey, 'XNA') =  COALESCE(W_BOOKED.purpleKey, 'XNA') 
    AND COALESCE(W_DECIDED.employmentstatus, 'XNA') =  COALESCE(W_BOOKED.employmentstatus, 'XNA') 
GROUP BY 1,2,3,4,5
)
, b1 as
(SELECT MONTH,dt, purpleKey, case when employmentstatus like 'Salaried' then 'Employed'
       When employmentstatus like 'Self Employed' then 'Self Employed'
       when employmentstatus like 'Remittance Beneficiary' then 'Remittance Beneficiary'
       when employmentstatus like 'Business Owner/Sole Proprietor' then 'Business Owner/Sole Proprietor'
      --  else 'Business Owner/Sole Proprietor, Housewife/House Husband, Overseas Filipino Worker, Retired, Student, Unemployed' 
      else 'Others'
       end  employmentstatus,date_flag,
max(merchantName) merchantName,
max(RBP_Flag)RBP_Flag,
SUM(Merchant) Merchant
, SUM(TSM) TSM
, min(pos_startofoperations)pos_startofoperations 
, SUM(Promoters) Promoters_assigned
, SUM(APP_STARTED) Application_cnt
, SUM(APPLICATION_BOOKING_CNT) APPLICATION_BOOKING_CNT
, sum(VAS_SOLD)VAS_SOLD
, sum(VAS_ELIGIBLE_BASE)VAS_ELIGIBLE_BASE
, sum(APPLICATION_APPROVED_CNT) APPLICATION_APPROVED_CNT
, sum(APPLICATION_DECISION_CNT) APPLICATION_DECISION_CNT
, round(case when sum(Items_Sold_Price) = 0 then 0 else sum(downPaymentAmount)/sum(Items_Sold_Price)end, 2) DOWNPAYMENT_RATE
, round(case when sum(APPLICATION_BOOKING_CNT) = 0 then 0 else sum(APPLICATION_BOOKED_TENURE)/sum(APPLICATION_BOOKING_CNT) end, 2) AVERAGE_LOANTERM
, round(case when sum(APPLICATION_BOOKING_CNT) = 0 then 0 else sum(BOOKED_VOLUME)/sum(APPLICATION_BOOKING_CNT)end, 2) AVERAGE_FINANCED_AMOUNT
, SUM(BOOKED_VOLUME) BOOKED_VOLUME
, CASE WHEN SUM(OBS_FPD30_CNT) = 0 THEN 0 ELSE ROUND(SUM(FPD30_CNT)/SUM(OBS_FPD30_CNT), 2) END FPD30
, CASE WHEN SUM(OBS_SPD30_CNT) = 0 THEN 0 ELSE ROUND(SUM(SPD30_CNT)/SUM(OBS_SPD30_CNT), 2) END SPD30
, CASE WHEN SUM(OBS_TPD30_CNT) = 0 THEN 0 ELSE ROUND(SUM(TPD30_CNT)/SUM(OBS_TPD30_CNT), 2) END TPD30
FROM b
GROUP BY 1,2,3,4,5
)
select MONTH, dt, purpleKey
, employmentstatus
, date_flag 
, max(case when date_flag = 'C' then '<2024-02-09'
           when date_flag = 'B' then '2024-02-09 - 2024-03-09'
           when date_flag = 'A' then '>2024-03-09' end) daterange
, max(merchantName)merchantName
, max(RBP_Flag) RBP_Flag
, max(Merchant) Merchant
, max(TSM) TSM
, min(pos_startofoperations)pos_startofoperations
, sum(Application_cnt) Application_cnt
, sum(APPLICATION_BOOKING_CNT) APPLICATION_BOOKING_CNT
, sum(VAS_SOLD) VAS_SOLD
, sum(VAS_ELIGIBLE_BASE) VAS_ELIGIBLE_BASE
, max(DOWNPAYMENT_RATE) DOWNPAYMENT_RATE
, max(case when DOWNPAYMENT_RATE = 0.0 then 'A. 0%'
           when DOWNPAYMENT_RATE > 0.0 and DOWNPAYMENT_RATE < 0.1 then 'B. 1%-10%'
           when DOWNPAYMENT_RATE >= 0.1 and DOWNPAYMENT_RATE < 0.2 then 'C. 10%-20%'
           when DOWNPAYMENT_RATE >= 0.2 and DOWNPAYMENT_RATE < 0.3 then 'D. 20%-30%'
           when DOWNPAYMENT_RATE >= 0.3 and DOWNPAYMENT_RATE < 0.5 then 'E. 30%-50%'
           when DOWNPAYMENT_RATE >= 0.5 then 'F. >50%' end
           ) downpaymentcat
, sum(BOOKED_VOLUME) BOOKED_VOLUME
from b1
group by 1,2,3,4,5"""

In [3]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID f1cc36c6-ab41-4a76-9c26-3b999dfff019 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [4]:
df.to_csv(r"D:\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Mikko\BNPL Report Redesign\Data\RBP_PILOT.csv", index = False)